## Educational Process Mining

https://archive-beta.ics.uci.edu/dataset/346/educational+process+mining+epm+a+learning+analytics+data+set

Educational Process Mining (EPM): A Learning Analytics Data Set. (2015). UCI Machine Learning Repository.

Educational Process Mining data set consists of student activity and outcome data for an e-learning simulation course in digital electronics. The experiments were carried out with a group of 115 first-year, undergraduate Engineering major students of the University of Genoa. The simulation environment, called Deeds (Digital Electronics Education and Design Suite), provides learning materials through specialized browsers and asks students to solve various problems with different levels of difficulty. For more information about the Deeds simulator see http://www.esng.dibe.unige.it/deeds/.

I chose this application because I wanted to work with student-level education data. This dataset contains exercise and activity inputs and interim and final grade outcome data. It is likely that my research for the LIS PhD program would also involve student-level data. My goal for this project is to learn how to visulaize and analyze this kind of data, which is time series where each input record has a start and end time and represents the amount of time the student spent on the recorded exercise and activity.

I have worked with education data in the past, but it was school-level aggregated data: School Performance Profile scores. I have domain knowledge (graduate coursework in Education and a PA state teaching certificate). I have not researched how to work with this data. 

I believe the data provided with this project is sufficient to answer the following questions:

* What factors influence student outcomes for this course?
* Is there a linear relationship between the total time spent and/or the number of sessions/exercises/activities completed and student outcomes? 
* Can student outcomes be predicted from time spent and/or the number of sessions/exercises/activities completed?

This notebook reads in all data, evaluates variables, creates new input variables, renames variables to simply exploration and modeling, and outputs the data to CSV files.

There are 6 session folders with student input data and two excel files with intermediate and final grades. Each session folder contains separate data sets for each student who participated in that session, either partially or fully. Each student data file contains fields for the session and student ID so each of these files were read in and concatenated to form one inputs data set. 

There are 13 columns and 227129 rows in the concatenated input data:

* Four ID columns as type integer - `session` and `student_id`
* Four object columns, two should be dates - `exercise`, `activity`, `start_time`, and `end_time`
* Seven integer columns - `idle_time`, `mouse_wheel`, `mouse_wheel_click`, `mouse_click_left`, `mouse_click_right`, `mouse_movement`, and `keystoke`

There are 6 columns and 115 rows in the intermediate grades data:

* One ID column as type integer - `Student Id`
* Five float columns for sessions 2 through 6 intermediate grades

There are 18 columns and 62 rows in the final grades data:

* One ID column as type interger - `Student ID`
* Seventeen float columns - 16 for the individual test question values and one for the total score 

There are no missing data in the input, intermediate grades, or final grades data. However, not all students participated in all sessions, exercises, activities, and final assessment so records do not exist in these instances.

The new input variables include

* `total_ms` is the difference in milliseconds between `start_time` and `end_time`.
* `activity_group` is the activity name, the first portion of the activity variable value, which consists of an activity and an associated exercise. Activities are 'Other', 'Aulaweb', 'Blank', 'Deeds', 'Study', 'Properties','Diagram', 'TextEditor', 'Study_Materials', 'Fsm_Related', 'FSM'
* `activity_exercise` is the exercise portion of the activity variable value.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

### Load the input data from github repository

Not every student completed every session. Get the list of files (named as student number which ranges from 1 to 115) from the local data and create a list from the results. Remove the value '1' from filenames_s1 because it was already loaded.

In [2]:
#import os
#os_path = "/Users/lisaover/Documents/EPM Dataset 2/Data/Processes/Session 6"
#filenames = os.listdir(os_path)
#filenames2 = [int(i) for i in filenames]
#filenames2.sort()
#print(len(filenames2))
#print(filenames2)

In [3]:
filenames_s1 = [2, 4, 5, 7, 9, 10, 11, 12, 14, 15, 17, 19, 20, 21, 22, 28, 30, 32, 34, 36, 37, 38, 39, 42, 43, 44, 46, 47, 49, 51, 52, 53, 54, 55, 56, 59, 62, 63, 66, 67, 68, 70, 71, 73, 74, 76, 78, 79, 80, 81, 82, 85, 86, 87, 88, 90, 91, 92, 93, 94, 96, 97, 98, 100, 101, 102, 104, 105, 108, 109, 110, 111, 112, 113, 114, 115]

In [4]:
filenames_s2 = [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 27, 28, 29, 30, 32, 34, 36, 38, 39, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 59, 61, 63, 66, 67, 70, 71, 72, 73, 74, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 107]

In [5]:
filenames_s3 = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 27, 28, 29, 30, 31, 32, 34, 35, 36, 38, 39, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 59, 61, 63, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 105, 106, 107]

In [6]:
filenames_s4 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]

In [7]:
filenames_s5 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104]

In [8]:
filenames_s6 = [1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 25, 27, 28, 29, 30, 32, 34, 35, 36, 37, 38, 39, 41, 42, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 102, 104]

In [9]:
path = 'https://raw.githubusercontent.com/lisaover/CMPINF2120_project/main/EPM%20Dataset%202/Data/Processes/Session%20'

In [10]:
inputs_init = pd.read_csv(path + "1/1", header=None)
inputs_init.columns = ['session','student_id','exercise','activity',
              'start_time','end_time','idle_time','mouse_wheel',
              'mouse_wheel_click','mouse_click_left','mouse_click_right',
              'mouse_movement','keystroke']
record_tally = len(inputs_init)
print(str(len(inputs_init)) + " records for Session 1, student " + str(inputs_init.student_id[0]) + "; " + str(record_tally) + " total records")

591 records for Session 1, student 1; 591 total records


In [11]:
def load_data(file_list, session):
    global path
    global record_tally
    global inputs_init
    for i in range(0,len(file_list)-1):
        file = path + str(session) + "/" + str(file_list[i])
        data = pd.read_csv(file, header=None)
        df = pd.DataFrame(data)
        df.columns = ['session','student_id','exercise','activity',
                  'start_time','end_time','idle_time','mouse_wheel',
                  'mouse_wheel_click','mouse_click_left','mouse_click_right',
                  'mouse_movement','keystroke']
        inputs_init = pd.concat([inputs_init, df], axis = 0)
        record_tally = record_tally + len(df)
        print(str(len(df)) + " records for Session " + str(session) + ", student " + str(df.student_id[0]) + "; " + str(record_tally) + " total records")

In [12]:
load_data(filenames_s1, 1)
load_data(filenames_s2, 2)
load_data(filenames_s3, 3)
load_data(filenames_s4, 4)
load_data(filenames_s5, 5)
load_data(filenames_s6, 6)

226 records for Session 1, student 2; 817 total records
574 records for Session 1, student 4; 1391 total records
567 records for Session 1, student 5; 1958 total records
325 records for Session 1, student 7; 2283 total records
507 records for Session 1, student 9; 2790 total records
354 records for Session 1, student 10; 3144 total records
457 records for Session 1, student 11; 3601 total records
697 records for Session 1, student 12; 4298 total records
380 records for Session 1, student 14; 4678 total records
485 records for Session 1, student 15; 5163 total records
184 records for Session 1, student 17; 5347 total records
191 records for Session 1, student 19; 5538 total records
543 records for Session 1, student 20; 6081 total records
351 records for Session 1, student 21; 6432 total records
423 records for Session 1, student 22; 6855 total records
353 records for Session 1, student 28; 7208 total records
599 records for Session 1, student 30; 7807 total records
600 records for Sess

In [13]:
inputs_init.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227129 entries, 0 to 644
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   session            227129 non-null  int64 
 1   student_id         227129 non-null  int64 
 2   exercise           227129 non-null  object
 3   activity           227129 non-null  object
 4   start_time         227129 non-null  object
 5   end_time           227129 non-null  object
 6   idle_time          227129 non-null  int64 
 7   mouse_wheel        227129 non-null  int64 
 8   mouse_wheel_click  227129 non-null  int64 
 9   mouse_click_left   227129 non-null  int64 
 10  mouse_click_right  227129 non-null  int64 
 11  mouse_movement     227129 non-null  int64 
 12  keystroke          227129 non-null  int64 
dtypes: int64(9), object(4)
memory usage: 24.3+ MB


In [14]:
inputs_init.shape

(227129, 13)

There are 13 columns:

* Four ID columns as type integer - session and student_id
* Four object columns, two should be dates - exercise, activity, start_time, and end_time
* Seven integer columns - idle_time, mouse_wheel, mouse_wheel_click, mouse_click_left, mouse_click_right, mouse_movement, and keystoke

There are no missing data.

In [15]:
inputs_df = inputs_init.copy()

#### Convert IDs and object variables to categorical type

In [16]:
inputs_df.session =  inputs_df.session.astype('object')
inputs_df.student_id =  inputs_df.student_id.astype('object')

In [17]:
inputs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227129 entries, 0 to 644
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   session            227129 non-null  object
 1   student_id         227129 non-null  object
 2   exercise           227129 non-null  object
 3   activity           227129 non-null  object
 4   start_time         227129 non-null  object
 5   end_time           227129 non-null  object
 6   idle_time          227129 non-null  int64 
 7   mouse_wheel        227129 non-null  int64 
 8   mouse_wheel_click  227129 non-null  int64 
 9   mouse_click_left   227129 non-null  int64 
 10  mouse_click_right  227129 non-null  int64 
 11  mouse_movement     227129 non-null  int64 
 12  keystroke          227129 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 24.3+ MB


In [18]:
inputs_df.isna().sum()

session              0
student_id           0
exercise             0
activity             0
start_time           0
end_time             0
idle_time            0
mouse_wheel          0
mouse_wheel_click    0
mouse_click_left     0
mouse_click_right    0
mouse_movement       0
keystroke            0
dtype: int64

In [19]:
inputs_df.session.nunique()

6

In [20]:
inputs_df.session.unique()

array([1, 2, 3, 4, 5, 6], dtype=object)

In [21]:
inputs_df.student_id.nunique()

113

In [22]:
inputs_df.student_id.unique()

array([1, 2, 4, 5, 7, 9, 10, 11, 12, 14, 15, 17, 19, 20, 21, 22, 28, 30,
       32, 34, 36, 37, 38, 39, 42, 43, 44, 46, 47, 49, 51, 52, 53, 54, 55,
       56, 59, 62, 63, 66, 67, 68, 70, 71, 73, 74, 76, 78, 79, 80, 81, 82,
       85, 86, 87, 88, 90, 91, 92, 93, 94, 96, 97, 98, 100, 101, 102, 104,
       105, 108, 109, 110, 111, 112, 113, 114, 3, 6, 8, 16, 18, 24, 27,
       29, 41, 48, 50, 61, 72, 83, 84, 95, 99, 103, 23, 25, 31, 35, 45,
       69, 75, 89, 106, 13, 26, 33, 40, 57, 58, 60, 64, 65, 77],
      dtype=object)

In [23]:
inputs_df.exercise.nunique()

30

In [24]:
inputs_df.exercise.unique()

array([' Es', ' Es_1_1', ' Es_1_2', ' Es_1_4', ' Es_1_3', ' Es_2_1',
       ' Es_2_2', ' Es_2_3', ' Es_2_4', ' Es_2_5', ' Es_2_6', ' Es_3_1',
       ' Es_3_2', ' Es_3_3', ' Es_3_4', ' Es_4_1', ' Es_4_2', ' Es_4_3',
       ' Es_4_4', ' Es_4_5', ' Es_5_2', ' Es_5_1', ' Es_5_3', ' Es_5_4',
       ' Es_6_1', ' Es_6_2', ' Es_6_3', ' Es_6_4', ' Es_6_6', ' Es_6_5'],
      dtype=object)

In [25]:
inputs_df['exercise'] = inputs_df['exercise'].str.strip()

In [26]:
inputs_df.activity.nunique()

99

In [27]:
inputs_df.activity.unique()

array([' Other', ' Aulaweb', ' Blank', ' Deeds', ' Study_Es_1_1',
       ' Deeds_Es_1_1', ' Properties', ' Diagram', ' Study_Es_1_2',
       ' Study_Es_1_4', ' TextEditor', ' TextEditor_Es_3_1',
       ' Deeds_Es_1_2', ' TextEditor_Es_1_2', ' TextEditor_Es_1_1',
       ' Study_Es_1_3', ' Deeds_Es_1_3', ' TextEditor_Es_1_3',
       ' Deeds_Es_1_4', ' TextEditor_Es_1_4', ' TextEditor_Es',
       ' Deeds_Es', ' Study_Materials', ' TextEditor_Es_4_1',
       ' Fsm_Related', ' TextEditor_Es_4_2', ' TextEditor_Es_4_3',
       ' TextEditor_Es_4_4', ' Study_Es_2_1', ' Deeds_Es_2_1',
       ' TextEditor_Es_2_1', ' Study_Es_2_2', ' Deeds_Es_2_2',
       ' TextEditor_Es_2_2', ' Study_Es_2_3', ' Deeds_Es_2_3',
       ' TextEditor_Es_2_3', ' Study_Es_2_4', ' TextEditor_Es_2_4',
       ' Deeds_Es_2_4', ' Study_Es_2_5', ' TextEditor_Es_2_5',
       ' Deeds_Es_2_5', ' Study_Es_2_6', ' TextEditor_Es_2_6',
       ' Deeds_Es_2_6', ' TextEditor_Es_3_2', ' TextEditor_Es_3_3',
       ' TextEditor_Es_3_4', '

In [28]:
inputs_df['activity'] = inputs_df['activity'].str.strip()

In [29]:
inputs_df.mouse_wheel.nunique()

460

In [30]:
inputs_df.mouse_wheel_click.nunique()

17

In [31]:
inputs_df.mouse_click_left.nunique()

394

In [32]:
inputs_df.mouse_click_right.nunique()

75

In [33]:
inputs_df.mouse_movement.nunique()

6221

In [34]:
inputs_df.keystroke.nunique()

808

#### Create column for total time between start and end times in milliseconds

This is to be consistent with variable `idle_time`, which is the idle time in milliseconds.

In [35]:
inputs_df['start_time'] = pd.to_datetime(inputs_df['start_time'])

In [36]:
inputs_df['end_time'] = pd.to_datetime(inputs_df['end_time'])

In [37]:
inputs_df['time_delta'] = inputs_df['end_time'] - inputs_df['start_time']

In [38]:
inputs_df['total_ms'] = inputs_df['time_delta'].dt.total_seconds() * 1000

In [39]:
inputs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227129 entries, 0 to 644
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype          
---  ------             --------------   -----          
 0   session            227129 non-null  object         
 1   student_id         227129 non-null  object         
 2   exercise           227129 non-null  object         
 3   activity           227129 non-null  object         
 4   start_time         227129 non-null  datetime64[ns] 
 5   end_time           227129 non-null  datetime64[ns] 
 6   idle_time          227129 non-null  int64          
 7   mouse_wheel        227129 non-null  int64          
 8   mouse_wheel_click  227129 non-null  int64          
 9   mouse_click_left   227129 non-null  int64          
 10  mouse_click_right  227129 non-null  int64          
 11  mouse_movement     227129 non-null  int64          
 12  keystroke          227129 non-null  int64          
 13  time_delta         227129 non-nu

#### Activity key

* Es: Exercise
* #: Number
* Deeds: Digital Electronics Education and Design Suite
* Diagram: Simulation Timing Diagram
* FSM: Finite State Machine Simulator

#### Split activity group from the exercise

In [40]:
inputs_df['activity'] = inputs_df['activity'].str.strip()

In [41]:
inputs_df[['activity_group','activity_exercise']] = inputs_df.activity.str.split( '_Es', expand=True )

In [42]:
inputs_df.activity_group.unique()

array(['Other', 'Aulaweb', 'Blank', 'Deeds', 'Study', 'Properties',
       'Diagram', 'TextEditor', 'Study_Materials', 'Fsm_Related', 'FSM',
       'FSM_Related'], dtype=object)

In [43]:
inputs_df['activity_group'] = inputs_df['activity_group'].str.strip()

In [44]:
inputs_df.activity_group.unique()

array(['Other', 'Aulaweb', 'Blank', 'Deeds', 'Study', 'Properties',
       'Diagram', 'TextEditor', 'Study_Materials', 'Fsm_Related', 'FSM',
       'FSM_Related'], dtype=object)

In [45]:
inputs_df['activity_exercise'] = inputs_df['activity_exercise'].str.replace("_", "Es_", 1)

In [46]:
inputs_df['activity_exercise'] = inputs_df['activity_exercise'].str.strip()

In [47]:
inputs_df.activity_exercise.unique()

array([None, 'Es_1_1', 'Es_1_2', 'Es_1_4', 'Es_3_1', 'Es_1_3', '',
       'Es_4_1', 'Es_4_2', 'Es_4_3', 'Es_4_4', 'Es_2_1', 'Es_2_2',
       'Es_2_3', 'Es_2_4', 'Es_2_5', 'Es_2_6', 'Es_3_2', 'Es_3_3',
       'Es_3_4', 'Es_4_5', 'Es_5_2', 'Es_5_1', 'Es_5_3', 'Es_5_4',
       'Es_6_1', 'Es_6_2', 'Es_6_3', 'Es_6_4', 'Es_6_6', 'Es_6_5'],
      dtype=object)

#### Check if `act_ex == exercise` for every row, and if so, delete `act_ex`. Otherwise keep `act_ex`.

In [48]:
activity_chk = inputs_df.copy()

In [49]:
activity_chk['chk_sum'] = np.where((activity_chk['exercise'] != activity_chk['activity_exercise']) & ~(activity_chk['activity_exercise'].isna()) & ~(activity_chk['activity_exercise'] == ''), 1, 0)

In [50]:
activity_chk.chk_sum.sum()

328

In [51]:
activity_chk.loc[activity_chk.chk_sum == 1]

,session,student_id,exercise,activity,start_time,end_time,idle_time,mouse_wheel,mouse_wheel_click,mouse_click_left,mouse_click_right,mouse_movement,keystroke,time_delta,total_ms,activity_group,activity_exercise,chk_sum
87,1,1,Es_1_1,TextEditor_Es_3_1,2014-02-10 11:58:18,2014-02-10 11:58:59,225830,5,0,20,4,1250,22,0 days 00:00:41,41000.0,TextEditor,Es_3_1,1
89,1,1,Es_1_1,TextEditor_Es_3_1,2014-02-10 11:59:04,2014-02-10 11:59:21,7357,0,0,8,0,1063,106,0 days 00:00:17,17000.0,TextEditor,Es_3_1,1
91,1,1,Es_1_1,TextEditor_Es_3_1,2014-02-10 11:59:33,2014-02-10 12:00:06,12660,5,0,48,0,2002,22,0 days 00:00:33,33000.0,TextEditor,Es_3_1,1
93,1,1,Es_1_1,TextEditor_Es_3_1,2014-02-10 12:00:14,2014-02-10 12:00:25,9681,0,0,10,0,375,18,0 days 00:00:11,11000.0,TextEditor,Es_3_1,1
103,1,1,Es_1_2,TextEditor_Es_3_1,2014-02-10 12:01:42,2014-02-10 12:01:42,0,0,0,2,0,48,0,0 days 00:00:00,0.0,TextEditor,Es_3_1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,6,89,Es_6_1,TextEditor_Es_3_1,2014-11-12 12:29:35,2014-11-12 12:29:36,282,0,0,2,0,50,0,0 days 00:00:01,1000.0,TextEditor,Es_3_1,1
129,6,89,Es_6_1,TextEditor_Es_3_1,2014-11-12 12:35:51,2014-11-12 12:37:23,4077151,0,0,1,0,75,0,0 days 00:01:32,92000.0,TextEditor,Es_3_1,1
132,6,89,Es_6_1,TextEditor_Es_3_1,2014-11-12 12:38:16,2014-11-12 12:38:17,16,0,0,2,0,42,0,0 days 00:00:01,1000.0,TextEditor,Es_3_1,1
146,6,89,Es_6_1,TextEditor_Es_3_1,2014-11-12 12:43:25,2014-11-12 12:44:03,672476,0,0,2,0,58,0,0 days 00:00:38,38000.0,TextEditor,Es_3_1,1


In [52]:
activity_chk.loc[activity_chk['activity_group'] == 'TextEditor'].shape

(41900, 18)

#### Delete `activity` column from inputs_df. Keep `activity_grp` and `activity_ex` as replacement (328 rows have `activity_ex` that is not equal to `exercise`).

In [53]:
inputs_out = inputs_df[['session', 'student_id', 'exercise','activity','activity_group', 
       'activity_exercise', 'start_time', 'end_time', 'total_ms', 'idle_time',
       'mouse_wheel', 'mouse_wheel_click', 'mouse_click_left', 'mouse_click_right', 
       'mouse_movement', 'keystroke']].\
       rename(columns={'session': 'sess', 'student_id': 'sid', 'exercise': 'exer', 'activity': 'actv', 
       'activity_group': 'actv_grp', 'activity_exercise': 'actv_exer', 'start_time': 'start_dt', 
       'end_time': 'end_dt', 'idle_time': 'idle_ms', 'mouse_wheel': 'mw', 'mouse_wheel_click': 'mwc', 
       'mouse_click_left': 'mcl', 'mouse_click_right': 'mcr', 'mouse_movement': 'mm', 'keystroke': 'ks' }).\
       copy()

In [54]:
inputs_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227129 entries, 0 to 644
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   sess       227129 non-null  object        
 1   sid        227129 non-null  object        
 2   exer       227129 non-null  object        
 3   actv       227129 non-null  object        
 4   actv_grp   227129 non-null  object        
 5   actv_exer  112679 non-null  object        
 6   start_dt   227129 non-null  datetime64[ns]
 7   end_dt     227129 non-null  datetime64[ns]
 8   total_ms   227129 non-null  float64       
 9   idle_ms    227129 non-null  int64         
 10  mw         227129 non-null  int64         
 11  mwc        227129 non-null  int64         
 12  mcl        227129 non-null  int64         
 13  mcr        227129 non-null  int64         
 14  mm         227129 non-null  int64         
 15  ks         227129 non-null  int64         
dtypes: datetime64[ns](2), f

In [55]:
inputs_out.to_csv('inputs_df.csv', index=False)

### Load the intermediate output data from github repository

In [56]:
interim_path = 'https://raw.githubusercontent.com/lisaover/CMPINF2120_project/main/EPM%20Dataset%202/Data/intermediate_grades.csv'

In [57]:
interim_init = pd.read_csv(interim_path)

In [58]:
interim_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Student Id  115 non-null    int64  
 1   Session 2   115 non-null    float64
 2   Session 3   115 non-null    float64
 3   Session 4   115 non-null    float64
 4   Session 5   115 non-null    float64
 5   Session 6   115 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 5.5 KB


In [59]:
interim_init.isna().sum()

Student Id    0
Session 2     0
Session 3     0
Session 4     0
Session 5     0
Session 6     0
dtype: int64

In [60]:
interim_df = interim_init.copy()

#### Melt the dataset and clean up the variable names

In [61]:
interim_df.columns

Index(['Student Id', 'Session 2', 'Session 3', 'Session 4', 'Session 5',
       'Session 6'],
      dtype='object')

In [62]:
interim_lf = interim_df.melt( id_vars=['Student Id'] ).copy()

In [63]:
interim_lf

,Student Id,variable,value
0,1,Session 2,5.0
1,2,Session 2,4.0
2,3,Session 2,3.5
3,4,Session 2,6.0
4,5,Session 2,5.0
...,...,...,...
570,111,Session 6,0.0
571,112,Session 6,0.0
572,113,Session 6,0.0
573,114,Session 6,0.0


In [64]:
interim_lf.variable.unique()

array(['Session 2', 'Session 3', 'Session 4', 'Session 5', 'Session 6'],
      dtype=object)

In [65]:
interim_lf['variable'] = interim_lf['variable'].str.replace("Session", "interim", regex=True)

In [66]:
interim_lf['variable'] = interim_lf['variable'].str.replace("[ ]", "_scr", regex=True)

In [67]:
interim_lf.variable.unique()

array(['interim_scr2', 'interim_scr3', 'interim_scr4', 'interim_scr5',
       'interim_scr6'], dtype=object)

#### Pivot interim_lf to wide dataset and ouput to CSV

In [68]:
interim_out = interim_lf.reset_index().loc[:, ['Student Id','variable','value']].\
                pivot(index=['Student Id'], columns='variable', values='value').\
                reset_index().copy()

interim_out

variable,Student Id,interim_scr2,interim_scr3,interim_scr4,interim_scr5,interim_scr6
0,1,5.0,0.0,4.5,4.0,2.25
1,2,4.0,3.5,4.5,4.0,1.00
2,3,3.5,3.5,4.5,4.0,0.00
3,4,6.0,4.0,5.0,3.5,2.75
4,5,5.0,4.0,5.0,4.0,2.75
...,...,...,...,...,...,...
110,111,3.0,0.0,0.0,3.0,0.00
111,112,0.0,0.0,0.0,0.0,0.00
112,113,0.0,0.0,0.0,0.0,0.00
113,114,0.0,0.0,0.0,0.0,0.00


#### Rename `Student ID` to match input dataset

In [69]:
interim_out.rename(columns={"Student Id": "sid"}, inplace=True)

In [70]:
interim_df.columns

Index(['Student Id', 'Session 2', 'Session 3', 'Session 4', 'Session 5',
       'Session 6'],
      dtype='object')

In [71]:
interim_out.columns

Index(['sid', 'interim_scr2', 'interim_scr3', 'interim_scr4', 'interim_scr5',
       'interim_scr6'],
      dtype='object', name='variable')

In [72]:
interim_out.to_csv('interim_df.csv', index=False)

### Load the final output data from github repository

In [73]:
final_path = 'https://raw.githubusercontent.com/lisaover/CMPINF2120_project/main/EPM%20Dataset%202/Data/final_grades.csv'

In [74]:
final_init = pd.read_csv(final_path)

In [75]:
final_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Student ID          62 non-null     int64  
 1   ES 1.1 
(2 points)  62 non-null     float64
 2   ES 1.2 
(3 points)  62 non-null     float64
 3   ES 2.1
(2 points)   62 non-null     float64
 4   ES 2.2
(3 points)   62 non-null     float64
 5   ES 3.1
(1 points)   62 non-null     float64
 6   ES 3.2
(2 points)   62 non-null     float64
 7   ES 3.3
(2 points)   62 non-null     float64
 8   ES 3.4
(2 points)   62 non-null     float64
 9   ES 3.5
(3 points)   62 non-null     float64
 10  ES 4.1
(15 points)  62 non-null     float64
 11  ES 4.2
(10 points)  62 non-null     float64
 12  ES 5.1
(2 points)   62 non-null     float64
 13  ES 5.2
(10 points)  62 non-null     float64
 14  ES 5.3
(3 points)   62 non-null     float64
 15  ES 6.1
(25 points)  62 non-null     float64
 16  ES 6.2
(15

In [76]:
final_init.isna().sum()

Student ID             0
ES 1.1 \n(2 points)    0
ES 1.2 \n(3 points)    0
ES 2.1\n(2 points)     0
ES 2.2\n(3 points)     0
ES 3.1\n(1 points)     0
ES 3.2\n(2 points)     0
ES 3.3\n(2 points)     0
ES 3.4\n(2 points)     0
ES 3.5\n(3 points)     0
ES 4.1\n(15 points)    0
ES 4.2\n(10 points)    0
ES 5.1\n(2 points)     0
ES 5.2\n(10 points)    0
ES 5.3\n(3 points)     0
ES 6.1\n(25 points)    0
ES 6.2\n(15 points)    0
TOTAL\n(100 points)    0
dtype: int64

In [77]:
final_df = final_init.copy()

#### Melt final dataset and clean up the column names 

Make the exercise (ES #.#) equal the execise variable in the input dataset

In [78]:
final_df.columns

Index(['Student ID', 'ES 1.1 \n(2 points)', 'ES 1.2 \n(3 points)',
       'ES 2.1\n(2 points)', 'ES 2.2\n(3 points)', 'ES 3.1\n(1 points)',
       'ES 3.2\n(2 points)', 'ES 3.3\n(2 points)', 'ES 3.4\n(2 points)',
       'ES 3.5\n(3 points)', 'ES 4.1\n(15 points)', 'ES 4.2\n(10 points)',
       'ES 5.1\n(2 points)', 'ES 5.2\n(10 points)', 'ES 5.3\n(3 points)',
       'ES 6.1\n(25 points)', 'ES 6.2\n(15 points)', 'TOTAL\n(100 points)'],
      dtype='object')

#### Melt final_df and clean up variable names

In [79]:
final_lf = final_df.melt( id_vars=['Student ID'], value_vars=['ES 1.1 \n(2 points)', 'ES 1.2 \n(3 points)',
       'ES 2.1\n(2 points)', 'ES 2.2\n(3 points)', 'ES 3.1\n(1 points)',
       'ES 3.2\n(2 points)', 'ES 3.3\n(2 points)', 'ES 3.4\n(2 points)',
       'ES 3.5\n(3 points)', 'ES 4.1\n(15 points)', 'ES 4.2\n(10 points)',
       'ES 5.1\n(2 points)', 'ES 5.2\n(10 points)', 'ES 5.3\n(3 points)',
       'ES 6.1\n(25 points)', 'ES 6.2\n(15 points)', 'TOTAL\n(100 points)'] ).copy()

In [80]:
final_lf

,Student ID,variable,value
0,1,ES 1.1 \n(2 points),2.0
1,2,ES 1.1 \n(2 points),2.0
2,4,ES 1.1 \n(2 points),2.0
3,5,ES 1.1 \n(2 points),2.0
4,7,ES 1.1 \n(2 points),2.0
...,...,...,...
1049,101,TOTAL\n(100 points),32.0
1050,102,TOTAL\n(100 points),31.5
1051,103,TOTAL\n(100 points),18.5
1052,104,TOTAL\n(100 points),92.0


In [81]:
final_lf[['question','point_str']] = final_lf.variable.str.split( '\n', expand=True )

In [82]:
final_lf['question'] = final_lf['question'].str.rstrip()

In [83]:
final_lf['question'] = final_lf['question'].str.replace("[S]", "s", regex=True)

In [84]:
final_lf['question'] = final_lf['question'].str.replace("[ ]", "_", regex=True)

In [85]:
final_lf['question'] = final_lf['question'].str.replace("[ .]", "q", regex=True)

In [86]:
final_lf.drop(['variable'], axis=1, inplace=True)

In [87]:
final_lf

,Student ID,value,question,point_str
0,1,2.0,Es_1q1,(2 points)
1,2,2.0,Es_1q1,(2 points)
2,4,2.0,Es_1q1,(2 points)
3,5,2.0,Es_1q1,(2 points)
4,7,2.0,Es_1q1,(2 points)
...,...,...,...,...
1049,101,32.0,TOTAL,(100 points)
1050,102,31.5,TOTAL,(100 points)
1051,103,18.5,TOTAL,(100 points)
1052,104,92.0,TOTAL,(100 points)


In [88]:
final_lf['question_points'] = final_lf.point_str.str.extract('(\d+)')

In [89]:
final_lf.drop(['point_str'], axis=1, inplace=True)

In [90]:
final_lf.loc[final_lf["question"] == "TOTAL", "question"] = "final_score"

In [91]:
final_lf

,Student ID,value,question,question_points
0,1,2.0,Es_1q1,2
1,2,2.0,Es_1q1,2
2,4,2.0,Es_1q1,2
3,5,2.0,Es_1q1,2
4,7,2.0,Es_1q1,2
...,...,...,...,...
1049,101,32.0,final_score,100
1050,102,31.5,final_score,100
1051,103,18.5,final_score,100
1052,104,92.0,final_score,100


#### Create lookup table for points

In [92]:
pts_final = pd.DataFrame(final_lf[['question','question_points']].\
                          groupby(['question'])['question_points'].\
                          agg(pd.Series.mode)).reset_index()

In [93]:
pts_final

,question,question_points
0,Es_1q1,2
1,Es_1q2,3
2,Es_2q1,2
3,Es_2q2,3
4,Es_3q1,1
5,Es_3q2,2
6,Es_3q3,2
7,Es_3q4,2
8,Es_3q5,3
9,Es_4q1,15


#### Pivot final_lf to wide dataset and output to CSV

In [94]:
final_lf.drop(['question_points'], axis=1, inplace=True)

In [95]:
final_lf

,Student ID,value,question
0,1,2.0,Es_1q1
1,2,2.0,Es_1q1
2,4,2.0,Es_1q1
3,5,2.0,Es_1q1
4,7,2.0,Es_1q1
...,...,...,...
1049,101,32.0,final_score
1050,102,31.5,final_score
1051,103,18.5,final_score
1052,104,92.0,final_score


In [96]:
final_out = final_lf.reset_index().loc[:, ['Student ID','value','question']].\
                pivot(index=['Student ID'], columns='question', values='value').\
                reset_index()


final_out

question,Student ID,Es_1q1,Es_1q2,Es_2q1,Es_2q2,Es_3q1,Es_3q2,Es_3q3,Es_3q4,Es_3q5,Es_4q1,Es_4q2,Es_5q1,Es_5q2,Es_5q3,Es_6q1,Es_6q2,final_score
0,1,2.0,3.0,1.0,0.5,1.0,2.0,2.0,2.0,3.0,15.0,10.0,2.0,10.0,3.0,25.0,13.0,94.5
1,2,2.0,3.0,2.0,0.5,1.0,2.0,0.0,2.0,3.0,15.0,2.0,0.0,5.0,1.5,5.0,0.0,44.0
2,4,2.0,3.0,1.0,0.5,1.0,2.0,0.0,2.0,0.0,3.0,4.0,0.0,1.5,0.0,5.0,5.0,30.0
3,5,2.0,3.0,2.0,1.5,1.0,2.0,2.0,2.0,3.0,3.0,2.0,1.5,9.0,1.5,2.0,1.0,38.5
4,7,2.0,3.0,1.0,1.5,1.0,2.0,2.0,2.0,3.0,15.0,10.0,1.0,2.5,0.0,20.0,12.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,101,2.0,2.0,1.0,2.0,1.0,2.0,1.5,2.0,2.0,5.0,1.0,0.0,0.0,1.5,4.0,5.0,32.0
58,102,2.0,3.0,1.0,1.5,1.0,0.0,0.0,2.0,3.0,12.0,3.0,0.0,0.0,0.0,2.0,1.0,31.5
59,103,2.0,3.0,1.0,1.5,1.0,2.0,0.0,2.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,18.5
60,104,2.0,3.0,1.0,1.5,1.0,2.0,2.0,2.0,3.0,15.0,9.0,2.0,8.0,3.0,24.5,13.0,92.0


#### Rename `Student ID` to match input dataset

In [97]:
final_out.rename(columns={"Student ID": "sid"}, inplace=True)

In [98]:
final_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sid          62 non-null     int64  
 1   Es_1q1       62 non-null     float64
 2   Es_1q2       62 non-null     float64
 3   Es_2q1       62 non-null     float64
 4   Es_2q2       62 non-null     float64
 5   Es_3q1       62 non-null     float64
 6   Es_3q2       62 non-null     float64
 7   Es_3q3       62 non-null     float64
 8   Es_3q4       62 non-null     float64
 9   Es_3q5       62 non-null     float64
 10  Es_4q1       62 non-null     float64
 11  Es_4q2       62 non-null     float64
 12  Es_5q1       62 non-null     float64
 13  Es_5q2       62 non-null     float64
 14  Es_5q3       62 non-null     float64
 15  Es_6q1       62 non-null     float64
 16  Es_6q2       62 non-null     float64
 17  final_score  62 non-null     float64
dtypes: float64(17), int64(1)
memory usage: 8.8 KB


In [99]:
pts_final.to_csv('final_points_lookup.csv', index=False)

In [100]:
final_out.to_csv('final_df.csv', index=False)

#### Create variable lookup datasets

In [101]:
inputs_df2 = inputs_df[['session', 'student_id', 'exercise', 'activity', 
                        'activity_group', 'activity_exercise', 'start_time', 'end_time',
                        'total_ms','idle_time', 'mouse_wheel', 'mouse_wheel_click',
                        'mouse_click_left', 'mouse_click_right', 'mouse_movement', 
                        'keystroke']].copy()

In [102]:
inputs_df2.columns

Index(['session', 'student_id', 'exercise', 'activity', 'activity_group',
       'activity_exercise', 'start_time', 'end_time', 'total_ms', 'idle_time',
       'mouse_wheel', 'mouse_wheel_click', 'mouse_click_left',
       'mouse_click_right', 'mouse_movement', 'keystroke'],
      dtype='object')

In [103]:
inputs_out.columns

Index(['sess', 'sid', 'exer', 'actv', 'actv_grp', 'actv_exer', 'start_dt',
       'end_dt', 'total_ms', 'idle_ms', 'mw', 'mwc', 'mcl', 'mcr', 'mm', 'ks'],
      dtype='object')

In [104]:
inputs_var_lookup = pd.DataFrame({'original_variable' : inputs_df2.columns,
                                  'new_variable' : inputs_out.columns})

In [105]:
inputs_var_lookup.to_csv('inputs_var_lookup.csv', index=False)

In [106]:
interim_df.columns

Index(['Student Id', 'Session 2', 'Session 3', 'Session 4', 'Session 5',
       'Session 6'],
      dtype='object')

In [107]:
interim_out.columns

Index(['sid', 'interim_scr2', 'interim_scr3', 'interim_scr4', 'interim_scr5',
       'interim_scr6'],
      dtype='object', name='variable')

In [108]:
interim_var_lookup = pd.DataFrame({'original_variable' : interim_df.columns,
                                  'new_variable' : interim_out.columns})

In [109]:
interim_var_lookup.to_csv('interim_var_lookup.csv', index=False)

In [110]:
final_df.columns

Index(['Student ID', 'ES 1.1 \n(2 points)', 'ES 1.2 \n(3 points)',
       'ES 2.1\n(2 points)', 'ES 2.2\n(3 points)', 'ES 3.1\n(1 points)',
       'ES 3.2\n(2 points)', 'ES 3.3\n(2 points)', 'ES 3.4\n(2 points)',
       'ES 3.5\n(3 points)', 'ES 4.1\n(15 points)', 'ES 4.2\n(10 points)',
       'ES 5.1\n(2 points)', 'ES 5.2\n(10 points)', 'ES 5.3\n(3 points)',
       'ES 6.1\n(25 points)', 'ES 6.2\n(15 points)', 'TOTAL\n(100 points)'],
      dtype='object')

In [111]:
final_out.columns

Index(['sid', 'Es_1q1', 'Es_1q2', 'Es_2q1', 'Es_2q2', 'Es_3q1', 'Es_3q2',
       'Es_3q3', 'Es_3q4', 'Es_3q5', 'Es_4q1', 'Es_4q2', 'Es_5q1', 'Es_5q2',
       'Es_5q3', 'Es_6q1', 'Es_6q2', 'final_score'],
      dtype='object', name='question')

In [112]:
final_var_lookup = pd.DataFrame({'original_variable' : final_df.columns,
                                  'new_variable' : final_out.columns})

In [113]:
final_var_lookup.to_csv('final_var_lookup.csv', index=False)